# Env Setting
아나콘다 준비 -> 터미널 열기  
conda create -n gpt python=3.10 anaconda  
conda activate gpt  
pip install -r requirements.txt  
jupyter notebook
  
Model download  
GPT4ALLModel_down_load url :   아래 코드 참고  
EmbeddingModel_down_load url :   https://huggingface.co/Pi3141/alpaca-native-7B-ggml/tree/397e872bf4c83f4c642317a5bf65ce84a105786e

*.bin으로 끝나는 파일 다운 받으면됨  
  
그리고 pdf file과 model path 자신의 path에 맞게 수정해주기 ~  
mac 최신 버전으로 업데이트도 해주기 ~

In [1]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import GPT4All
from langchain.callbacks import StdOutCallbackHandler
# function for loading only TXT files
from langchain.document_loaders import TextLoader
# text splitter for create chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
# to be able to load the pdf files
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
# Vector Store Index to create our database about our knowledge
from langchain.indexes import VectorstoreIndexCreator
# LLamaCpp embeddings from the Alpaca model
from langchain.embeddings import LlamaCppEmbeddings
# FAISS  library for similaarity search
from langchain.vectorstores.faiss import FAISS
import os  #for interaaction with the files
import datetime

# Model Load
- GPT4ALL model
- Embedding Model

Down load GPT4ALL model 
- 아래 코드가 모델 다운하는 코드임 !
- 다운 다되면 모델 path가 보임!, 그것을 path에 추가해서 활용해 주자
- 다운중에 에러나면 모델이 제대로 작동하지 않음. 에러가 난다면 해당 파일을 삭제후에 다시 다운하기!

In [2]:
import gpt4all

gptj = gpt4all.GPT4All("ggml-gpt4all-j-v1.3-groovy")

Found model file.
gptj_model_load: loading model from '/Users/youngyong/.cache/gpt4all/ggml-gpt4all-j-v1.3-groovy.bin' - please wait ...
gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 5401.45 MB
gptj_model_load: kv self size  =  896.00 MB
gptj_model_load: ................................... done
gptj_model_load: model size =  3609.38 MB / num tensors = 285


In [4]:
# 모델 다운한 path넣기 ~
gpt4all_path = '/Users/youngyong/.cache/gpt4all/ggml-gpt4all-j-v1.3-groovy.bin' 
llama_path = './models/ggml-model-q4_0.bin' 
# Calback manager for handling the calls with  the model
handler = StdOutCallbackHandler()
# create the GPT4All llm object
llm = GPT4All(model=gpt4all_path, callbacks=[handler])

Found model file.
gptj_model_load: loading model from '/Users/youngyong/.cache/gpt4all/ggml-gpt4all-j-v1.3-groovy.bin' - please wait ...
gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 5401.45 MB
gptj_model_load: kv self size  =  896.00 MB
gptj_model_load: ................................... done
gptj_model_load: model size =  3609.38 MB / num tensors = 285


In [5]:
# create the embedding object
embeddings = LlamaCppEmbeddings(model_path=llama_path)

llama.cpp: loading model from ./models/ggml-model-q4_0.bin
llama.cpp: can't use mmap because tensors are not aligned; convert to new format to avoid this
llama_model_load_internal: format     = 'ggml' (old version with low tokenizer quality and no mmap support)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size = 4113748.20 KB
llama_model_load_internal: mem required  = 5809.33 MB (+ 2052.00 MB per state)
...................................................................................................
.
llam

# Data Load & Embedding

- 함수 선언
> split_chunks : 사이즈별로 파일 나누기  
> create_index : text의 chunk를 embedding화  
> similarity_search : 질문과 가장 유사한 word 뽑아내기  

In [4]:
# Split text 
def split_chunks(sources):
    chunks = []
    splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)
    for chunk in splitter.split_documents(sources):
        chunks.append(chunk)
    return chunks


def create_index(chunks):
    texts = [doc.page_content for doc in chunks]
    metadatas = [doc.metadata for doc in chunks]

    search_index = FAISS.from_texts(texts, embeddings, metadatas=metadatas)

    return search_index


def similarity_search(query, index):
    # k is the number of similarity searched that matches the query
    # default is 4
    matched_docs = index.similarity_search(query, k=3) 
    sources = []
    for doc in matched_docs:
        sources.append(
            {
                "page_content": doc.page_content,
                "metadata": doc.metadata,
            }
        )

    return matched_docs, sources

- PDF load -> Text 변환 -> Chunk 사이즈로 나누기 -> 임베딩 벡터 변환

In [9]:
# get the list of pdf files from the docs directory into a list  format
pdf_folder_path = './docs'
doc_list = [s for s in os.listdir(pdf_folder_path) if s.endswith('.pdf')]
num_of_docs = len(doc_list)

# create a loader for the PDFs from the path
general_start = datetime.datetime.now() #not used now but useful
print("starting the loop...")
loop_start = datetime.datetime.now() #not used now but useful
print("generating fist vector database and then iterate with .merge_from")
# loader = PyPDFLoader(os.path.join(pdf_folder_path, doc_list[0]))
# docs = loader.load()
# chunks = split_chunks(docs)
# db0 = create_index(chunks)
print("Main Vector database created. Start iteration and merging...")
for i in range(num_of_docs):
    print(doc_list[i])
    print('embedding file nm : ',doc_list[i].split('.')[0])
    print(f"loop position {i}")
    loader = PyPDFLoader(os.path.join(pdf_folder_path, doc_list[i]))
    start = datetime.datetime.now() #not used now but useful
    docs = loader.load()
    chunks = split_chunks(docs)
    dbi = create_index(chunks)
    print("start merging with db0...")
    dbi.save_local(doc_list[i].split('.')[0])
    end = datetime.datetime.now() #not used now but useful
    elapsed = end - start #not used now but useful
    #total time
    print(f"completed in {elapsed}")
    print("-----------------------------------")
loop_end = datetime.datetime.now() #not used now but useful
loop_elapsed = loop_end - loop_start #not used now but useful
print(f"All documents processed in {loop_elapsed}")
print(f"the daatabase is done with {num_of_docs} subset of db index")
print("-----------------------------------")


starting the loop...
generating fist vector database and then iterate with .merge_from
Main Vector database created. Start iteration and merging...
AirPodsPro.pdf
embedding file nm :  AirPodsPro
loop position 0



llama_print_timings:        load time =  3899.34 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time = 78401.35 ms /   221 tokens (  354.76 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time = 78528.02 ms

llama_print_timings:        load time =  3899.34 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time = 66403.24 ms /   194 tokens (  342.28 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time = 66512.55 ms

llama_print_timings:        load time =  3899.34 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time = 43129.09 ms /   125 tokens (  345.03 ms per token)
llama_print_timings

start merging with db0...
completed in 0:23:17.574004
-----------------------------------
AirPods.pdf
embedding file nm :  AirPods
loop position 1



llama_print_timings:        load time =  3899.34 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time = 94901.77 ms /   224 tokens (  423.67 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time = 95137.95 ms

llama_print_timings:        load time =  3899.34 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time = 81623.55 ms /   211 tokens (  386.84 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time = 81797.32 ms

llama_print_timings:        load time =  3899.34 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time = 53851.40 ms /   142 tokens (  379.24 ms per token)
llama_print_timings


llama_print_timings:        load time =  3899.34 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time = 115111.71 ms /   298 tokens (  386.28 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time = 115300.15 ms

llama_print_timings:        load time =  3899.34 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time = 82779.93 ms /   237 tokens (  349.28 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time = 82918.09 ms

llama_print_timings:        load time =  3899.34 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time = 88237.68 ms /   251 tokens (  351.54 ms per token)
llama_print_timin

start merging with db0...
completed in 0:46:23.977213
-----------------------------------
AirPodsMax.pdf
embedding file nm :  AirPodsMax
loop position 2



llama_print_timings:        load time =  3899.34 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time = 67794.80 ms /   200 tokens (  338.97 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time = 67906.52 ms

llama_print_timings:        load time =  3899.34 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time = 71081.89 ms /   208 tokens (  341.74 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time = 71198.69 ms

llama_print_timings:        load time =  3899.34 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time = 65205.08 ms /   152 tokens (  428.98 ms per token)
llama_print_timings

start merging with db0...
completed in 0:24:50.582939
-----------------------------------
All documents processed in 1:34:32.203047
the daatabase is done with 3 subset of db index
-----------------------------------



llama_print_timings:        load time =  3899.34 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time = 23934.29 ms /    70 tokens (  341.92 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time = 23976.22 ms


### Faiss를 통한 query와 파일간의 유사도가 높은 문장 뽑아오기
- 에어팟, 에어팟프로, 에어팟맥스 pdf파일의 text와 유사도 검사 진행  
> 1. query = "product's brief explanation, does not contain the word wikipedia"  
> 2. 각 파일의 유사도가 높은 문장을 가져옴 (아래에서 확인)

문제? : 정의한 query의 문제인지 가져오는 문장이 품질이 좋지 않은 이슈 존재

In [5]:
pdf_folder_path = './docs'
doc_list = [s for s in os.listdir(pdf_folder_path) if s.endswith('.pdf')]

dic = {}
for file_nm in doc_list:
    file_nm = file_nm.split('.')[0]
    index = FAISS.load_local(file_nm, embeddings)
    # Load our local index vector db
#     query = "Product description".format(file_nm)
    query = "product's brief explanation, does not contain the word wikipedia"
    docs = index.similarity_search(query)
    # Get the matches best 3 results - defined in the function k=3
    print(f"The question is: {query}")
    print("Here the result of the semantic search on the index, without GPT4All..")
#     print(docs)
    dic[file_nm] = {}
    dic[file_nm]['information'] = ''.join(docs[1].page_content.split('\n'))
    print(dic[file_nm]['information'])


llama_print_timings:        load time =  3827.46 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =  6442.64 ms /    16 tokens (  402.67 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =  6453.17 ms


The question is: product's brief explanation, does not contain the word wikipedia
Here the result of the semantic search on the index, without GPT4All..
accelerometers  and optical senso rs that can detec t presses on the stemand in-ear placement, and automati c pausing when they are taken out ofthe ears. Control by tapping is repla ced by pressing a force sensor on thestems.[ 7 ] They are rated IPX4  for water resistance.[ 3]The AirPods Pro use the H1 chip  also found in the second and third-generation AirPods, that supports hands-free "Hey Siri". They have activenoise cancellation , accomplished by microphones detecting outside sound



llama_print_timings:        load time =  3827.46 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =  5348.84 ms /    16 tokens (  334.30 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =  5357.45 ms


The question is: product's brief explanation, does not contain the word wikipedia
Here the result of the semantic search on the index, without GPT4All..
with each other. Having earphones in is generally perceived as a "do not disturb" signal, and some AirPods usersuse this fact to strategically avoid awkward small talk.[ 6 8 ] But in the collaborative workplace, constant use ofAirPods has sometimes been interpreted as a sign of disrespect to fellow co-workers, causing conflict in theoffice.[ 6 9 ]Another prominent criticism was a problem that caused the charging case battery to deplete at a rapid rate
The question is: product's brief explanation, does not contain the word wikipedia
Here the result of the semantic search on the index, without GPT4All..
23. 5. 30. 오후  6:12 AirPods Max - W ikipediahttps://en.wikipedia.or g/wiki/AirPods_Max 1/6AirPods MaxAirPods Max in Space GrayDeveloper Apple Inc.ManufacturerLuxshare (on contract),Goertek (on contract)ProductfamilyAirPodsType Wireless he


llama_print_timings:        load time =  3827.46 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =  5364.73 ms /    16 tokens (  335.30 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =  5373.74 ms


In [6]:
context = """
The format of the Context is in the order of 'product name' '-' 'description of the product'
{} - {}
The format of the Context is in the order of 'product name' '-' 'description of the product'
{} - {}
The format of the Context is in the order of 'product name' '-' 'description of the product'
{} - {}
""".format('AirPods',dic['AirPods']['information'], 'AirPodsPro',dic['AirPodsPro']['information'], 'AirPodsMax',dic['AirPodsMax']['information'])
context

'\nThe format of the Context is in the order of \'product name\' \'-\' \'description of the product\'\nAirPods - with each other. Having earphones in is generally perceived as a "do not disturb" signal, and some AirPods usersuse this fact to strategically avoid awkward small talk.[ 6 8 ] But in the collaborative workplace, constant use ofAirPods has sometimes been interpreted as a sign of disrespect to fellow co-workers, causing conflict in theoffice.[ 6 9 ]Another prominent criticism was a problem that caused the charging case battery to deplete at a rapid rate\nThe format of the Context is in the order of \'product name\' \'-\' \'description of the product\'\nAirPodsPro - accelerometers  and optical senso rs that can detec t presses on the stemand in-ear placement, and automati c pausing when they are taken out ofthe ears. Control by tapping is repla ced by pressing a force sensor on thestems.[ 7 ] They are rated IPX4  for water resistance.[ 3]The AirPods Pro use the H1 chip  also fo

# 질답
- Template
> - context : Faiss를 통해 뽑혀온 txt  
> - question : 나의 질문  
> - answer : LLM의 답변  

In [9]:
# # Load our local index vector db
# index = FAISS.load_local("my_faiss_index", embeddings)

# create the prompt template
template = """
Please use the following context to answer questions.

Context: {context}
---
Question: {question}
Answer: Let's think step by step. Hashtags on Instagram have the format of '#words' """

# Hardcoded question
question = "Express airpods, airpods pro, airpods max product features like Instagram hashtags"
# instantiating the prompt template and the GPT4All chain
prompt = PromptTemplate(template=template, input_variables=["context", "question"]).partial(context=context)
handler = StdOutCallbackHandler()
llm_chain = LLMChain(prompt=prompt, llm=llm, callbacks=[handler])
# Print the result
print(llm_chain.run(question))






> Entering new LLMChain chain...
Prompt after formatting:

Please use the following context to answer questions.

Context: 
The format of the Context is in the order of 'product name' '-' 'description of the product'
AirPods - with each other. Having earphones in is generally perceived as a "do not disturb" signal, and some AirPods usersuse this fact to strategically avoid awkward small talk.[ 6 8 ] But in the collaborative workplace, constant use ofAirPods has sometimes been interpreted as a sign of disrespect to fellow co-workers, causing conflict in theoffice.[ 6 9 ]Another prominent criticism was a problem that caused the charging case battery to deplete at a rapid rate
The format of the Context is in the order of 'product name' '-' 'description of the product'
AirPodsPro - accelerometers  and optical senso rs that can detec t presses on the stemand in-ear placement, and automati c pausing when they are taken out ofthe ears. Control by tapping is repla ced by pressing a force sen

In [10]:
# # Load our local index vector db
# index = FAISS.load_local("my_faiss_index", embeddings)

# create the prompt template
template = """
Please use the following context to answer questions.

Context: {context}
---
Question: {question}
Answer: Let's think step by step."""

# Hardcoded question
question = "Express airpods, airpods pro, airpods max product features like Instagram hashtags"
# instantiating the prompt template and the GPT4All chain
prompt = PromptTemplate(template=template, input_variables=["context", "question"]).partial(context=context)
handler = StdOutCallbackHandler()
llm_chain = LLMChain(prompt=prompt, llm=llm, callbacks=[handler])
# Print the result
print(llm_chain.run(question))






> Entering new LLMChain chain...
Prompt after formatting:

Please use the following context to answer questions.

Context: 
The format of the Context is in the order of 'product name' '-' 'description of the product'
AirPods - with each other. Having earphones in is generally perceived as a "do not disturb" signal, and some AirPods usersuse this fact to strategically avoid awkward small talk.[ 6 8 ] But in the collaborative workplace, constant use ofAirPods has sometimes been interpreted as a sign of disrespect to fellow co-workers, causing conflict in theoffice.[ 6 9 ]Another prominent criticism was a problem that caused the charging case battery to deplete at a rapid rate
The format of the Context is in the order of 'product name' '-' 'description of the product'
AirPodsPro - accelerometers  and optical senso rs that can detec t presses on the stemand in-ear placement, and automati c pausing when they are taken out ofthe ears. Control by tapping is repla ced by pressing a force sen

In [11]:
# # Load our local index vector db
# index = FAISS.load_local("my_faiss_index", embeddings)

# create the prompt template
template = """
Please use the following context to answer questions.

Context: {context}
---
Question: {question}
Answer: Let's think step by step. """

# Hardcoded question
question = "Express AirPods, AirPodsPro, AirPodsMax product features like Instagram hashtags"
# instantiating the prompt template and the GPT4All chain
prompt = PromptTemplate(template=template, input_variables=["context", "question"]).partial(context=context)
handler = StdOutCallbackHandler()
llm_chain = LLMChain(prompt=prompt, llm=llm, callbacks=[handler])
# Print the result
print(llm_chain.run(question))






> Entering new LLMChain chain...
Prompt after formatting:

Please use the following context to answer questions.

Context: 
The format of the Context is in the order of 'product name' '-' 'description of the product'
AirPods - with each other. Having earphones in is generally perceived as a "do not disturb" signal, and some AirPods usersuse this fact to strategically avoid awkward small talk.[ 6 8 ] But in the collaborative workplace, constant use ofAirPods has sometimes been interpreted as a sign of disrespect to fellow co-workers, causing conflict in theoffice.[ 6 9 ]Another prominent criticism was a problem that caused the charging case battery to deplete at a rapid rate
The format of the Context is in the order of 'product name' '-' 'description of the product'
AirPodsPro - accelerometers  and optical senso rs that can detec t presses on the stemand in-ear placement, and automati c pausing when they are taken out ofthe ears. Control by tapping is repla ced by pressing a force sen

In [13]:
# # Load our local index vector db
# index = FAISS.load_local("my_faiss_index", embeddings)

# create the prompt template
template = """
Please use the following context to answer questions.

Context: {context}
---
Question: {question}
Answer: Let's think step by step. """

# Hardcoded question
question = """
Can you give me some simple keywords to describe the product below?
Products are separated by ','. 
AirPods, AirPodsPro, AirPodsMax """

# instantiating the prompt template and the GPT4All chain
prompt = PromptTemplate(template=template, input_variables=["context", "question"]).partial(context=context)
handler = StdOutCallbackHandler()
llm_chain = LLMChain(prompt=prompt, llm=llm, callbacks=[handler])
# Print the result
print(llm_chain.run(question))






> Entering new LLMChain chain...
Prompt after formatting:

Please use the following context to answer questions.

Context: 
The format of the Context is in the order of 'product name' '-' 'description of the product'
AirPods - with each other. Having earphones in is generally perceived as a "do not disturb" signal, and some AirPods usersuse this fact to strategically avoid awkward small talk.[ 6 8 ] But in the collaborative workplace, constant use ofAirPods has sometimes been interpreted as a sign of disrespect to fellow co-workers, causing conflict in theoffice.[ 6 9 ]Another prominent criticism was a problem that caused the charging case battery to deplete at a rapid rate
The format of the Context is in the order of 'product name' '-' 'description of the product'
AirPodsPro - accelerometers  and optical senso rs that can detec t presses on the stemand in-ear placement, and automati c pausing when they are taken out ofthe ears. Control by tapping is repla ced by pressing a force sen

In [14]:


# # Load our local index vector db
# index = FAISS.load_local("my_faiss_index", embeddings)

# create the prompt template
template = """
Please use the following context to answer questions.

Context: {context}
---
Question: {question}
Answer: Let's think step by step. """

# Hardcoded question
question = """Explain the product description of AirPods, AirPods Pro, and AirPods Max with simple keywords"""

# instantiating the prompt template and the GPT4All chain
prompt = PromptTemplate(template=template, input_variables=["context", "question"]).partial(context=context)
handler = StdOutCallbackHandler()
llm_chain = LLMChain(prompt=prompt, llm=llm, callbacks=[handler])
# Print the result
print(llm_chain.run(question))






> Entering new LLMChain chain...
Prompt after formatting:

Please use the following context to answer questions.

Context: 
The format of the Context is in the order of 'product name' '-' 'description of the product'
AirPods - with each other. Having earphones in is generally perceived as a "do not disturb" signal, and some AirPods usersuse this fact to strategically avoid awkward small talk.[ 6 8 ] But in the collaborative workplace, constant use ofAirPods has sometimes been interpreted as a sign of disrespect to fellow co-workers, causing conflict in theoffice.[ 6 9 ]Another prominent criticism was a problem that caused the charging case battery to deplete at a rapid rate
The format of the Context is in the order of 'product name' '-' 'description of the product'
AirPodsPro - accelerometers  and optical senso rs that can detec t presses on the stemand in-ear placement, and automati c pausing when they are taken out ofthe ears. Control by tapping is repla ced by pressing a force sen

In [17]:


# # Load our local index vector db
# index = FAISS.load_local("my_faiss_index", embeddings)

# create the prompt template
template = """
---
Question: {question}
Answer: Let's think step by step. """

# Hardcoded question
question = """Explain the product description of AirPods, AirPods Pro, and AirPods Max with simple keywords"""

# instantiating the prompt template and the GPT4All chain
prompt = PromptTemplate(template=template, input_variables=["question"])
handler = StdOutCallbackHandler()
llm_chain = LLMChain(prompt=prompt, llm=llm, callbacks=[handler])
# Print the result
print(llm_chain.run(question))






> Entering new LLMChain chain...
Prompt after formatting:

---
Question: Explain the product description of AirPods, AirPods Pro, and AirPods Max with simple keywords
Answer: Let's think step by step. 

 1. Product name (Air Pods): The company that produces these earphones is Apple Inc., which sells a variety of products under its umbrella such as iPhones or iPads in the market segment for personal electronics devices like portable media players, computers and gaming systems. So to make it clear what this product stands for: "EarPods", meaning small electronic device with wires inside your ears that transmit sound (like music) from an outside source into them through a tiny opening on top of each earphone which then sends the signal wirelessly via Bluetooth 4 or later versions up and down to some sort of receiver, amplifier if needed." 2. Product description: "EarPods" are small electronic devices that fit comfortably in your ears with wires inside for sound transmission from an outs

---

여러 질문을 해보았지만 오히려 context를 제외한 답변이 제일 만족스러웟음.  

context의 품질이 좋지 않았나 ? 더 많은 실험이 필요해보임.  